In [1]:
import os
import sys
import logging
import time
import numpy as np
from functools import wraps
import types

In [2]:
class SampleClass:
    
    def __init__(self):
        pass
    
    def whatisSelf(self):
        return self


In [3]:
sample_instance  = SampleClass()

### Everytime a method is called, the instance itself is automatically passed to the method (function), which gives the function access to the instance. The self argument is the created instance of the class.

In [4]:
sample_instance

In [5]:
sample_instance.whatisSelf()

In [6]:
sample_instance == sample_instance.whatisSelf()

True

## Decorators

In [7]:
def outer_function(msg):
    
    def inner_function():
        return msg
    
    return inner_function

In [8]:
hi_func = outer_function('Hi')
bye_func =  outer_function('Bye')

In [9]:
hi_func()

'Hi'

In [10]:
bye_func()

'Bye'

In [11]:
def decorator_function(original_function):
    
    def wrapper_function():
        print(f'wrapper executed this before the {original_function.__name__} function')
        return original_function()
    
    return wrapper_function

In [12]:
@decorator_function
def greet():
    print('Hey there!')

In [13]:
greet()

wrapper executed this before the greet function
Hey there!


In [14]:
def greet():
    print('Hey there!')

In [15]:
greet = decorator_function(greet)

In [16]:
greet()

wrapper executed this before the greet function
Hey there!


### Decorator which can take in arguments of the function

In [17]:
@decorator_function
def greet(name, timeofday):
    print(f'How are you this {timeofday}, {name}?')

In [18]:
greet('Abhi', 'morning')

TypeError: wrapper_function() takes 0 positional arguments but 2 were given

In [19]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print(f'wrapper executed this before the {original_function.__name__} function')
        return original_function(*args, **kwargs)    
    
    return wrapper_function

In [20]:
@decorator_function
def greet(name, timeofday):
    print(f'How are you this {timeofday}, {name}?')

In [21]:
greet('Abhi', 'morning')

wrapper executed this before the greet function
How are you this morning, Abhi?


In [22]:
def greet(name, timeofday):
    print(f'How are you this {timeofday}, {name}?')

In [23]:
greet = decorator_function(greet)

In [24]:
greet('Abhi', 'morning')

wrapper executed this before the greet function
How are you this morning, Abhi?


## Similiar to function, you could use a class to decorate as well

In [25]:
class DecoratorClass:
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        print(f'__call__ method of {self.__class__.__name__} class ',
              f'executed this before the function {self.func.__name__}', sep='\n')
        return self.func(*args, **kwargs)

In [26]:
def greet(name, timeofday):
    print(f'How are you this {timeofday}, {name}?')

In [27]:
greet = DecoratorClass(greet)

In [26]:
greet('Abhi', 'morning')

__call__ method of DecoratorClass class 
executed this before the function greet
How are you this morning, Abhi?


In [23]:
@DecoratorClass
def greet(name, timeofday):
    print(f'How are you this {timeofday}, {name}?')

In [24]:
greet('Abhi', 'morning')

__call__ method of DecoratorClass class 
executed this before the function greet
How are you this morning, Abhi?


## Making instances callable

In [25]:
class TestClass:
    def __init__(self):
        pass
    
    def __call__(self, *args, **kwargs):
        
        return f'calling with {args} and {kwargs}!'

In [26]:
test_instance = TestClass()

In [27]:
test_instance('arg1', 'arg2', kwarg1='kwarg1', kwarg2='kwarg2')

"calling with ('arg1', 'arg2') and {'kwarg1': 'kwarg1', 'kwarg2': 'kwarg2'}!"

In [28]:
def my_logger(orig_func):
    logging.basicConfig(filename=f'output/logging/{orig_func.__name__}.log', level=logging.INFO)
    
    def wrapper(*args, **kwargs):
        logging.info(f'Ran with args : {args} and kwargs :{kwargs}')
        
        return orig_func(*args, **kwargs)
    
    return wrapper
        
    

In [29]:
@my_logger
def greet(name, timeofday):
    print(f'How are you this {timeofday}, {name}?')

In [30]:
greet('Abhi', 'morning')
greet('Akku', 'evening')

How are you this morning, Abhi?
How are you this evening, Akku?


In [31]:
fact.__name__

NameError: name 'fact' is not defined

In [32]:
!cat output/logging/greet.log

INFO:root:Ran with args : ('Abhi', 'morning') and kwargs :{}
INFO:root:Ran with args : ('Akku', 'evening') and kwargs :{}


In [33]:
def my_timer(orig_func):
    
    def wrapper(*args, **kwargs):
        t_start = time.time()
        orig_func(*args, **kwargs)
        return f'The function {orig_func.__name__} with args {args}'\
                +f' and kwargs {kwargs} took {np.round(time.time()- t_start, 5)}s'
    
    return wrapper

In [34]:
@my_timer
def fact(num):
    if num == 0:
        return 1
    
    return num*fact(num-1)

In [35]:
fact(500)

'The function fact with args (500,) and kwargs {} took 0.01993s'

In [36]:
fact.__name__

'wrapper'

In [37]:
def my_timer(orig_func):
    
    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t_start = time.time()
        orig_func(*args, **kwargs)
        return f'The function {orig_func.__name__} with args {args}'\
                +f' and kwargs {kwargs} took {np.round(time.time()- t_start, 5)}s'
    
    return wrapper

In [38]:
@my_timer
def fact(num):
    if num == 0:
        return 1
    
    return num*fact(num-1)

In [39]:
fact.__name__

'fact'

## Be careful while returning the result of a recursive function in a decorator

In [ ]:
from time import time

def decorator_function(func):
    def wrapper(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        return f'Elapsed: {t2-t1}', result ### Here lies the problem
    
    return wrapper
    
@decorator_function    
def fact(num):
    if num == 0:
        return 1
    else:
        return num*fact(num-1)
    
fact(10)

## More than one decorator for a function

In [40]:
@my_timer
@my_logger
def fact(num):
    if num == 0:
        return 1
    
    return num*fact(num-1)

#### Wrong function name in output statemnt as my_logger returns function name as wrapper

In [41]:
fact(500)

'The function wrapper with args (500,) and kwargs {} took 0.02895s'

In [42]:
def fact(num):
    if num == 0:
        return 1
    
    return num*fact(num-1)

In [43]:
fact = my_timer(my_logger(fact))

In [44]:
fact(500)

'The function wrapper with args (500,) and kwargs {} took 0.02991s'

In [45]:
@my_logger
@my_timer
def fact(num):
    if num == 0:
        return 1
    
    return num*fact(num-1)

#### Right function name in statement but wrong log file created (should have been)
## Looks like some problem with logging and jupyter

In [46]:
fact(500)

'The function fact with args (500,) and kwargs {} took 0.03682s'

In [47]:
os.listdir('output/logging/')

['greet.log']

## Decorators with arguments for multiple situation based wrapping needs

In [48]:
def prefix_decorator(prefix):
    
    def decorator_function(orig_func):
        print(prefix)
        
        @wraps(orig_func)
        def wrapper(*args, **kwargs):
            print('Executed before the function')
            orig_func(*args, **kwargs)
            print('Executed after the function')

        return wrapper
        
    return decorator_function

### Problem with putting any execution of print other than wrapper

In [49]:
@prefix_decorator('LOG:')
def greet(name, timeofday):
    print(f'How are you this {timeofday} {name}?')

LOG:


In [50]:
def prefix_decorator(prefix):
    
    def decorator_function(orig_func):
        
        @wraps(orig_func)
        def wrapper(*args, **kwargs):
            print(prefix)
            print('Executed before the function')
            orig_func(*args, **kwargs)
            print('Executed after the function')

        return wrapper
        
    return decorator_function

In [51]:
@prefix_decorator('LOG:')
def greet(name, timeofday):
    print(f'How are you this {timeofday} {name}?')

In [52]:
greet('Abhi', 'morning')

LOG:
Executed before the function
How are you this morning Abhi?
Executed after the function


In [53]:
@prefix_decorator('DEBUG:')
def greet(name, timeofday):
    print(f'How are you this {timeofday} {name}?')

In [54]:
greet('Abhi', 'morning')

DEBUG:
Executed before the function
How are you this morning Abhi?
Executed after the function


## Doing something before every method in a class

In [60]:
class User(object):
    
    def __init__(self, age):
        self.age = age
        self.isBanned = 'No'
    
    def __getattribute__(self, attr):
        method = object.__getattribute__(self, attr)
        if not method:
            raise Exception(f"Method {attr} is not implemented!")
        
        if type(method) == types.MethodType and method.__name__ != '__null__':
            if self.isBanned == 'Yes':
                print('You have been banned by the admin!')
                return self.__null__
            
        return method
    
    def __null__(self):
        pass
    
    def createAccount(self):
        return 'In progress but atleast you are not banned!'

In [61]:
print()

In [62]:
me = User(25)

In [63]:
me.isBanned = 'Yes'

In [64]:
me.createAccount()

You have been banned by the admin!


## Getters, setters and deleters

In [109]:
class Employee():
    
    def __init__(self, first, last):
        self.first = first
        self.last = last
        self.email = f'{self.first}.{self.last}@email.com'
        
    @property
    def fullname(self):
        return f'{self.first} {self.last}'
        
    def __repr__(self):
        return 'An instance of class {self.__class__.__name__}'
    

emp_1 = Employee('Abhi', 'Bha')



In [110]:
emp_1.fullname

'Abhi Bha'

In [111]:
emp_1.fullname = 'Abhishek Bhatia'

AttributeError: can't set attribute

In [112]:
emp_1.__setattr__('fullname', 'Abhishek Bhatia')

AttributeError: can't set attribute

In [ ]:
class Employee():
    
    def __init__(self, first, last):
        self.first = first
        self.last = last
        self.email = f'{self.first}.{self.last}@email.com'
        
    @property
    def fullname(self):
        return f'{self.first} {self.last}'
    
    @fullname.setter
    def fullname(self, name):
        self.first, self.last = name.split
        
    def __repr__(self):
        return 'An instance of class {self.__class__.__name__}'
    

## Lambda function vs normal function

In [70]:
def add(*args):
    '''Adds the integer arguments provided'''
    
    return sum(args)

add(1, 2, 3)

6

In [71]:
print(f'name of the function add is {add.__name__}')
print(f'help text of the function add is {help(add)}')

name of the function add is add
Help on function add in module __main__:

add(*args)
    Adds the integer arguments provided

help text of the function add is None


### calling help on a fuction just prints the doc and returns None

In [82]:
add = lambda *args: args
add(1, 2, 3)

(1, 2, 3)

In [83]:
print(f'name of the function add is {add.__name__}')
print(f'help text of the function add is {help(add)}')

name of the function add is <lambda>
Help on function <lambda> in module __main__:

<lambda> lambda *args

help text of the function add is None


#### Let's fix that

In [84]:
add.__name__ = 'add'
add.__doc__ = 'Adds the integer arguments provided'

In [86]:
print(f'name of the function add is {add.__name__}')
print(f'help text of the function add is {help(add)}')


name of the function add is add
Help on function add in module __main__:

add(*args)
    Adds the integer arguments provided

help text of the function add is None


#### You can't use pass in a lambda function

In [87]:
add = lambda *args: pass

SyntaxError: invalid syntax (<ipython-input-87-3d839d0152ce>, line 1)